In [ ]:
! pip install gradio
!pip install openai-whisper
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 15.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.1 MB/s eta

In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa
import whisper
import gradio as gr

def load_wav2vec_model(language):
    """Load the appropriate Wav2Vec2 model for English and Hindi."""
    model_mapping = {
        "english": "facebook/wav2vec2-large-960h",
        "hindi": "theainerd/Wav2Vec2-large-xlsr-hindi"
    }

    if language not in model_mapping:
        raise ValueError("Unsupported language for Wav2Vec2. Use Whisper for Tamil, Telugu, Malayalam.")

    model_name = model_mapping[language]
    print(f"Loading Wav2Vec2 model: {model_name}")
    processor = Wav2Vec2Processor.from_pretrained(model_name)
    model = Wav2Vec2ForCTC.from_pretrained(model_name)
    return processor, model

def transcribe_wav2vec(audio_path, language):
    """Transcribe speech using Wav2Vec2 model."""
    processor, model = load_wav2vec_model(language)

    speech_array, sampling_rate = librosa.load(audio_path, sr=16000)
    input_values = processor(speech_array, sampling_rate=16000, return_tensors="pt", padding=True).input_values

    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription

def transcribe_whisper(audio_path):
    """Transcribe speech using Whisper model for Tamil, Telugu, Malayalam."""
    model = whisper.load_model("medium")
    result = model.transcribe(audio_path)
    return result["text"]

def transcribe(audio, language):
    """Gradio function to handle audio input and transcribe."""
    if audio is None:
        return "No audio file provided."

    print(f"Received audio file: {audio}")

    if language in ["english", "hindi"]:
        transcription = transcribe_wav2vec(audio, language)
    elif language in ["tamil", "telugu", "malayalam"]:
        transcription = transcribe_whisper(audio)
    else:
        return "Unsupported language."

    return f"Transcription: {transcription}"

iface = gr.Interface(
    fn=transcribe,
    inputs=[
        gr.Audio(type="filepath"),
        gr.Dropdown(["english", "hindi", "tamil", "malayalam", "telugu"], label="Select Language")
    ],
    outputs="text",
    title="Multilingual Speech-to-Text",
    description="Upload an audio file, select a language, and get the transcribed text."
)

if __name__ == "__main__":
    print("Launching Gradio interface...")
    iface.launch(debug=True)

Launching Gradio interface...
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://992a34359d8a245c42.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Received audio file: /tmp/gradio/71d5b98e4e1d0186b0e0f9cab4187a8cb6c20fcfbb27bd29c7ad9b78bb16c109/174-50561-0002.flac
Loading Wav2Vec2 model: facebook/wav2vec2-large-960h


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]